In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import learning_curve

# Logistic Regression

Load data

In [ ]:
x_trn = None #TODO
y_trn = None
x_tst = None

Set random seed

In [ ]:
rng = np.random.RandomState(0)

Create cross-validation generator

In [ ]:
cvg = StratifiedKFold(n_splits=5, shuffle=False, random_state=rng)

Create an instance of the classifier we want to use

In [ ]:
clf = LogisticRegression(penalty='l2', random_state=rng, solver='liblinear')

Take a look at the parameters we'll need to create a grid for

In [ ]:
clf.get_params(deep=False)

Create parameter grid

In [ ]:
param_grid = {}#TODO
param_grid = ParameterGrid(param_grid)

Create successive halving grid search cross-validation estimator

In [ ]:
search = HalvingGridSearchCV(estimator=clf, 
                             param_grid=param_grid, 
                             factor=3, 
                             cv=cvg, 
                             random_state=rng).fit(x_trn, y_trn)

Create and save successive halving heatmap https://scikit-learn.org/stable/auto_examples/model_selection/plot_successive_halving_heatmap.html#

In [ ]:
#TODO

Plot and save candidate scores over successive halving iterations https://scikit-learn.org/stable/auto_examples/model_selection/plot_successive_halving_iterations.html#

In [ ]:
#TODO

Plot and save feature importance scores to determine top features https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html

In [ ]:
#TODO

Plot and save ROC curve https://scikit-learn.org/stable/modules/generated/sklearn.metrics.RocCurveDisplay.html#sklearn.metrics.RocCurveDisplay.from_predictions

In [ ]:
#TODO
#use RocCurveDisplay.from_predictions(y_true, y_pred)

Set classifier parameters to optimal values identified by cross-validation and fit on entire training set, then output predictions on test set

In [ ]:
clf.set_params(search.best_params_) #check value, might need to join with params from block 4
clf.fit(x_trn, y_trn)
#we need to decide what to do with these predictions, since it's an unlabeled test set.